In [12]:
# imports and froms
import pandas as pd
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [13]:
# load CSVs into dataframes
income_csv = 'income.csv'
income_df = pd.read_csv(income_csv, encoding = 'utf8')
income_df.head()

,State,State In.,City,Average,Median,Unnamed: 5,Unnamed: 6
0,California,CA,San Francisco,"$100,856.92","$84,786.00",NaN,NaN
1,California,CA,San Diego,"$84,908.78","$75,188.00",NaN,NaN
2,California,CA,Los Angeles,"$61,724.56","$39,202.50",NaN,NaN
3,New York,NY,New York,"$88,873.63","$68,128.00",NaN,NaN
4,Massachusetts,MA,Boston,"$78,482.50","$51,575.00",NaN,NaN


In [14]:
# income_df.drop(['id', 'State_Code', 'County', 'Place', 'Primary', 'Area_Code', 'ALand', 'AWater'], axis=1)

In [15]:
host = '127.0.0.1' 
user = 'root'
password = 'Bacon0531'
db = 'income_db'
charset = 'utf8'
port = 3306
cursorclass = pymysql.cursors.DictCursor

In [16]:
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

In [17]:
# create database
sql_create = (f'CREATE DATABASE IF NOT EXISTS {db}')  
cursor_object.execute(sql_create)

# show databases
sql_query = 'SHOW DATABASES'
cursor_object.execute(sql_query)
db_list = cursor_object.fetchall()

for database in db_list:
    print(database)

{'Database': 'etl_db'}
{'Database': 'income_db'}
{'Database': 'information_schema'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'sys'}


/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1007, "Can't create database 'income_db'; database exists")
  self._do_get_result()


In [18]:
# reestablish pymysql connection with database specified
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db, 
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

In [19]:
# create tables
query_list = [
    'CREATE TABLE IF NOT EXISTS cities(City VARCHAR(32), State VARCHAR(32), Mean INT, Median INT)'
]

for query in query_list:
    cursor_object.execute(query)

# show tables
sql_query = 'SHOW TABLES'   
cursor_object.execute(sql_query)
rows = cursor_object.fetchall()

for row in rows:
    print(row)

{'Tables_in_income_db': 'cities'}


/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1050, "Table 'cities' already exists")
  self._do_get_result()


In [20]:
# establish sqlalchemy connection
con = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8', echo=False)

# load data into tables
income_df.to_sql(name='cities', con=con, if_exists='replace', index=False)

In [21]:
# reestablish pymysql connection after loading data
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db, 
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

# # show data in tables
# query_list = [
#     'SELECT * FROM cities LIMIT 10',
# ]

# for query in query_list:
#     cursor_object.execute(query)
#     rows = cursor_object.fetchall()

#     for row in rows:
#         print(row)
        
#     print(end='\n')

In [22]:
# terminate pymysql connection
connection.close()